In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/subtitles/eng_subtitles_database.db


In [2]:
import sqlite3
from tqdm import tqdm

In [3]:
connt = sqlite3.connect('/kaggle/input/subtitles/eng_subtitles_database.db')
cursor = connt.cursor()
cursor.execute("SELECT name From sqlite_master WHERE type = 'table'")
print(cursor.fetchall())

[('zipfiles',)]


In [4]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
  print(col[1])

num
name
content


In [5]:
%time df = pd.read_sql_query("""SELECT * FROM zipfiles""",connt)
df.head()

CPU times: user 1.56 s, sys: 2.64 s, total: 4.2 s
Wall time: 43.9 s


,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [7]:
df = df.sample(frac = 0.3)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 62413 to 64236
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      24749 non-null  int64 
 1   name     24749 non-null  object
 2   content  24749 non-null  object
dtypes: int64(1), object(2)
memory usage: 773.4+ KB


In [8]:
import zipfile
import io

count = 0

def decode_method(binary_data):
  global count
  count += 1
  with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f,'r') as zip_file:
      subtitle_content = zip_file.read(zip_file.namelist()[0])
  return subtitle_content.decode('latin-1')

In [9]:
tqdm.pandas()
%time df['file_content'] = df['content'].progress_apply(decode_method)
df.head()

100%|██████████| 24749/24749 [00:11<00:00, 2242.46it/s]

CPU times: user 10.3 s, sys: 756 ms, total: 11.1 s
Wall time: 11.1 s


,num,name,content,file_content
62413,9439876,allo.allo.s03.e05.pretty.maids.all.in.a.row.(1...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x07A\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver..."
5589,9205308,its.always.sunny.in.philadelphia.s04.e07.who.p...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xa6\x8d\x...,"ï»¿1\r\n00:00:02,202 --> 00:00:03,613\r\n<i>Gu..."
42819,9355247,a.storm.for.christmas.s01.e05.episode.1.5.(202...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00R\x08\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:09,880\r\n<i>â..."
70664,9472339,dam.s02.e05.the.witch.hunt.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xddZ\x9aV...,"1\r\n00:00:00,000 --> 00:00:04,000\r\n- (myste..."
18383,9255307,bring.it.on.cheer.or.die.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00<\x9b\x99V...,"ï»¿1\r\n00:00:02,698 --> 00:00:04,917\r\n[dram..."


In [10]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
from typing import final
import re
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')
def text_process(text):
    clean_text = text.lower()
    final_text = re.sub(r'[^a-z\s]', '', clean_text)
    tokenizer = model.tokenizer
    tokens = tokenizer.tokenize(final_text)
    return " ".join(tokens)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
tqdm.pandas()
%time df['file_content'] = df['file_content'].progress_apply(text_process)

100%|██████████| 24749/24749 [12:42<00:00, 32.46it/s]

CPU times: user 12min 42s, sys: 2.25 s, total: 12min 44s
Wall time: 12min 42s


In [13]:
data = df['file_content']
data.shape

(24749,)

In [15]:
def chunk_text_document(text, chunk_size=1200, overlap=150):
    chunks = []
    start_index = 0
    end_index = chunk_size
    while start_index < len(text):
        end_index = min(end_index, len(text))
        chunk = text[start_index:end_index]
        chunks.append(chunk)
        start_index += chunk_size - overlap
        end_index += chunk_size - overlap
    return chunks

chunked_text_documents = []

for text_document in data:
    chunks = chunk_text_document(text_document)
    chunked_text_documents.extend(chunks)



In [16]:
chunked_text = chunked_text_documents

In [18]:
batch_size = 10000

with tqdm(total=len(chunked_text_documents)) as pbar:
    chunk_embeddings = []

    for i in range(0, len(chunked_text_documents), batch_size):
        batch_embeddings = []

        batch_chunks = chunked_text_documents[i:i+batch_size]

        batch_embeddings.extend(model.encode(batch_chunks))

        chunk_embeddings.extend(batch_embeddings)

        pbar.update(len(batch_chunks))

chunk_embeddings_array = np.array(chunk_embeddings)


  0%|          | 0/685441 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

  1%|▏         | 10000/685441 [02:08<2:24:51, 77.71it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

  3%|▎         | 20000/685441 [04:17<2:22:33, 77.79it/s]

...

 98%|█████████▊| 670000/685441 [2:22:47<03:16, 78.43it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

 99%|█████████▉| 680000/685441 [2:24:55<01:09, 78.35it/s]

Batches:   0%|          | 0/171 [00:00<?, ?it/s]

100%|██████████| 685441/685441 [2:26:05<00:00, 78.20it/s]


In [19]:
chunked_text_embeddings = chunk_embeddings_array
len(chunked_text_embeddings), len(chunked_text)

(685441, 685441)

In [23]:
flattened_embeddings = np.array(chunked_text_embeddings).flatten()
chunk_text = pd.DataFrame({"text": chunked_text})
chunk_text_vector = pd.DataFrame({"embedding": flattened_embeddings})

In [24]:
chunk_text.to_csv('chunk_text.csv', index = False)
chunk_text_vector.to_csv('chunk_text_vector.csv', index = False)